In [1]:
!pip install praw
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime


### Methodology 1. Reddit praw api

In [2]:
### put your personal info 
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='')

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

In [8]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [2]:
ls

MadeMeSmile_submissions.zst
Positivity_submissions.zst
anxiety_depression_submissions.parquet
anxiety_depression_submissions_preprocessed.parquet
depression_submissions.zst
happy_submissions.parquet
happy_submissions.zst
happy_submissions_preprocessed.parquet
positivity_submissions.parquet
positivity_submissions_preprocessed.parquet
reddit_data_preprocessing_AnxietyDepression.ipynb
reddit_data_preprocessing_Mademesmile.ipynb
reddit_data_preprocessing_Positivity.ipynb
reddit_data_preprocessing_depression.ipynb
reddit_data_preprocessing_happy.ipynb
reddit_data_preprocessing_v1.ipynb


In [5]:
## step1. load downloaded submission reddit dump 

In [3]:
mademesmile_submissions =\
    pd.read_json('MadeMeSmile_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 100000)

In [4]:
# convert the timestamp to utc
mademesmile_submissions['created_utc'] =\
    mademesmile_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [5]:
## step2. download the file into parquet file to speed up data load later stage

In [6]:
mademesmile_submissions.to_parquet('mademesmile_submissions.parquet', engine='pyarrow')

In [7]:
## step3. read as parquet file 

In [8]:
mademesmile_submissions = pd.read_parquet('mademesmile_submissions.parquet',engine='pyarrow')
mademesmile_submissions

,downs,link_flair_text,distinguished,media,url,link_flair_css_class,id,edited,num_reports,created_utc,banned_by,name,subreddit,title,author_flair_text,is_self,author,media_embed,permalink,author_flair_css_class,selftext,domain,num_comments,likes,clicked,thumbnail,saved,subreddit_id,ups,approved_by,score,selftext_html,created,hidden,over_18,gilded,retrieved_on,report_reasons,secure_media,secure_media_embed,stickied,mod_reports,user_reports,hide_score,from_kind,from_id,quarantine,archived,from,preview,post_hint,locked,contest_mode,spoiler,suggested_sort,brand_safe,author_cakeday,thumbnail_width,can_gild,thumbnail_height,view_count,is_video,approved_at_utc,banned_at_utc,can_mod_post,is_crosspostable,num_crossposts,parent_whitelist_status,whitelist_status,crosspost_parent,crosspost_parent_list,is_reddit_media_domain,pinned,subreddit_type,mod_note,mod_reason_by,mod_reason_title,no_follow,send_replies,subreddit_subscribers,author_flair_background_color,author_flair_richtext,author_flair_template_id,author_flair_text_color,author_flair_type,category,content_categories,is_original_content,link_flair_richtext,link_flair_text_color,link_flair_type,media_only,post_categories,pwls,removal_reason,rte_mode,visited,wls,link_flair_background_color,link_flair_template_id,previous_visits,media_metadata,is_meta,subreddit_name_prefixed,author_created_utc,author_fullname,author_patreon_flair,gildings,is_robot_indexable,all_awardings,total_awards_received,allow_live_comments,discussion_type,author_premium,awarders,removed_by,removed_by_category,treatment_tags
0,2.0,None,None,None,http://www.youtube.com/watch?v=CHD6AX6RuO8,None,y0vrd,0,NaN,2012-08-10 23:59:22,NaN,t3_y0vrd,MadeMeSmile,"Might not be for everyone, but the vocalist ba...",None,False,LoudMimeDave,"{'content': None, 'height': None, 'media_domai...",/r/MadeMeSmile/comments/y0vrd/might_not_be_for...,None,,youtube.com,1,NaN,0.0,default,0.0,t5_2uqcm,7.0,NaN,5,None,1.344647e+09,0.0,False,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None
1,1.0,None,None,"{'oembed': {'author_name': 'Rick Mereki', 'aut...",http://vimeo.com/27246366,None,y0vni,0,NaN,2012-08-10 23:57:22,NaN,t3_y0vni,MadeMeSmile,MOVE by Rick Mereki,None,False,Joe_fh,"{'content': '&lt;iframe src=""http://player.vim...",/r/MadeMeSmile/comments/y0vni/move_by_rick_mer...,None,,vimeo.com,1,NaN,0.0,http://d.thumbs.redditmedia.com/eujGmh5LiDTya9...,0.0,t5_2uqcm,17.0,NaN,16,None,1.344647e+09,0.0,False,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None
2,0.0,None,None,"{'oembed': {'author_name': 'explotdavis', 'aut...",http://www.youtube.com/watch?v=nucSvl7VXVM,None,y0ukn,0,NaN,2012-08-10 23:37:15,NaN,t3_y0ukn,MadeMeSmile,Great Song and a Smile-Worthy Video,None,False,Spankapotamus,"{'content': '&lt;iframe width=""600"" height=""33...",/r/MadeMeSmile/comments/y0ukn/great_song_and_a...,None,,youtube.com,0,NaN,0.0,http://a.thumbs.redditmedia.com/tNWmzU5d5DKz_f...,0.0,t5_2uqcm,4.0,NaN,4,None,1.344645e+09,0.0,False,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None
3,3.0,None,None,{'oembed': {'author_name': 'ForTheLoveofChl

In [9]:
## step4. feature selection : select only necessary columns

In [10]:
mademesmile_submissions =\
    mademesmile_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
mademesmile_submissions


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,MadeMeSmile,LoudMimeDave,False,None,"Might not be for everyone, but the vocalist ba...",,http://www.youtube.com/watch?v=CHD6AX6RuO8,2012-08-10 23:59:22
1,MadeMeSmile,Joe_fh,False,None,MOVE by Rick Mereki,,http://vimeo.com/27246366,2012-08-10 23:57:22
2,MadeMeSmile,Spankapotamus,False,None,Great Song and a Smile-Worthy Video,,http://www.youtube.com/watch?v=nucSvl7VXVM,2012-08-10 23:37:15
3,MadeMeSmile,Intolight,False,None,This made me smile along with everyone else in...,,http://www.youtube.com/watch?v=vUJIgkPoiBg,2012-08-10 23:32:20
4,MadeMeSmile,stego_,False,None,This never fails me,,http://i.imgur.com/8E4Cc.jpg,2012-08-10 23:06:39
...,...,...,...,...,...,...,...,...
99995,MadeMeSmile,[deleted],False,None,"In times like this, we need to share the love....",[deleted],https://gfycat.com/genuineshycockerspaniel,2019-12-18 17:24:23
99996,MadeMeSmile,mtimetraveller,False,None,Dad Washes Baby In The Most Wholesome Way! ❤,,https://gfycat.com/craftyzestybellfrog,2019-12-18 17:25:13
99997,MadeMeSmile,[deleted],False,None,Plane passenger gives first-class seat to 88-y...,[deleted],https://i.redd.it/5i31s3crgf541.jpg,2019-12-18 17:34:33
99998,MadeMeSmile,IncandescentlyHappy,False,None,Stevie Ticks meets the boys next door,,https://i.redd.it/2m0w0c2vhf541.jpg,2019-12-18 17:40:44


In [11]:
## step5. basic pre-processing 

In [12]:
# remove blank body posts
mademesmile_submissions = mademesmile_submissions[mademesmile_submissions['selftext'].apply(lambda x : len(x)!=0)]
mademesmile_submissions


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
34,MadeMeSmile,ProneToStanding,False,None,There's something about videos of dancing fool...,...so here's two examples!\n\nhttp://www.youtu...,http://www.reddit.com/r/MadeMeSmile/comments/y...,2012-08-10 18:57:24
48,MadeMeSmile,photo,False,None,Subreddit for 19 minutes!,This should catch on. 22 subscribers.,http://www.reddit.com/r/MadeMeSmile/comments/y...,2012-08-10 18:13:59
209,MadeMeSmile,gbaker386,False,None,"Gained faith in humanity, and it made my day",Sorry this is going to be a bit of a lengthy s...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-10-04 19:15:33
249,MadeMeSmile,[deleted],False,None,Helicopter rescues a RC plane from a tree,http://www.youtube.com/watch?v=ffEYqGGYXRk&amp...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-10-29 21:05:05
253,MadeMeSmile,K_Lobstah,False,None,"I normally hate cross-posting, but I thought y...",[Link to original post here](http://www.reddit...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-11-02 19:44:20
...,...,...,...,...,...,...,...,...
99990,MadeMeSmile,[deleted],False,None,Spread the cheer ✨,[deleted],https://i.redd.it/peyohqpq9f541.png,2019-12-18 16:55:13
99992,MadeMeSmile,[deleted],False,None,This is Myosin protein pulling an endorphin al...,[deleted],https://i.redd.it/mh7s3b50af541.gif,2019-12-18 16:57:22
99995,MadeMeSmile,[deleted],False,None,"In times like this, we need to share the love....",[deleted],https://gfycat.com/genuineshycockerspaniel,2019-12-18 17:24:23
99997,MadeMeSmile,[deleted],False,None,Plane passenger gives first-class seat to 88-y...,[deleted],https://i.redd.it/5i31s3crgf541.jpg,2019-12-18 17:34:33


In [13]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
mademesmile_submissions[mademesmile_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
5654,MadeMeSmile,[deleted],False,None,These toothbrush holders are REALLY FUCKING EX...,[deleted],http://i.imgur.com/q2C8XZM.jpg,2015-08-26 05:55:11
5655,MadeMeSmile,[deleted],False,None,Two buddies who served in Vietnam reunite afte...,[deleted],https://www.youtube.com/watch?v=iIOlV5rFHag&am...,2015-08-26 14:05:00
5658,MadeMeSmile,[deleted],False,None,Baby Penguin Meets a Human,[deleted],http://i.imgur.com/8q83o3k.gifv,2015-08-26 15:56:17
5660,MadeMeSmile,[deleted],False,None,Best Cute Dog Haircut Ever,[deleted],https://www.youtube.com/watch?v=pXoBkIUGsus,2015-08-26 16:38:26
5661,MadeMeSmile,[deleted],False,None,"Boop, Boop, Boop, Boop.......Boop",[deleted],https://youtu.be/KKrkYN1TTAw?t=19,2015-08-26 17:34:57
...,...,...,...,...,...,...,...,...
99990,MadeMeSmile,[deleted],False,None,Spread the cheer ✨,[deleted],https://i.redd.it/peyohqpq9f541.png,2019-12-18 16:55:13
99992,MadeMeSmile,[deleted],False,None,This is Myosin protein pulling an endorphin al...,[deleted],https://i.redd.it/mh7s3b50af541.gif,2019-12-18 16:57:22
99995,MadeMeSmile,[deleted],False,None,"In times like this, we need to share the love....",[deleted],https://gfycat.com/genuineshycockerspaniel,2019-12-18 17:24:23
99997,MadeMeSmile,[deleted],False,None,Plane passenger gives first-class seat to 88-y...,[deleted],https://i.redd.it/5i31s3crgf541.jpg,2019-12-18 17:34:33


In [14]:
mademesmile_submissions = mademesmile_submissions[~mademesmile_submissions['selftext'].isin(opt_out_list)]
print(len(mademesmile_submissions))
mademesmile_submissions

1946


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
34,MadeMeSmile,ProneToStanding,False,None,There's something about videos of dancing fool...,...so here's two examples!\n\nhttp://www.youtu...,http://www.reddit.com/r/MadeMeSmile/comments/y...,2012-08-10 18:57:24
48,MadeMeSmile,photo,False,None,Subreddit for 19 minutes!,This should catch on. 22 subscribers.,http://www.reddit.com/r/MadeMeSmile/comments/y...,2012-08-10 18:13:59
209,MadeMeSmile,gbaker386,False,None,"Gained faith in humanity, and it made my day",Sorry this is going to be a bit of a lengthy s...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-10-04 19:15:33
249,MadeMeSmile,[deleted],False,None,Helicopter rescues a RC plane from a tree,http://www.youtube.com/watch?v=ffEYqGGYXRk&amp...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-10-29 21:05:05
253,MadeMeSmile,K_Lobstah,False,None,"I normally hate cross-posting, but I thought y...",[Link to original post here](http://www.reddit...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-11-02 19:44:20
...,...,...,...,...,...,...,...,...
99730,MadeMeSmile,jansbri,False,None,We Organized Carolers to Perform ‘All I Want F...,&amp;#x200B;\n\n[ ](https://reddit.com/link/eb...,https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-16 17:32:20
99814,MadeMeSmile,pokey_bum_wannk,False,None,"So, we have this Columbian fella working with us.",He doesn't speak much English but he's a real ...,https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-17 09:26:36
99831,MadeMeSmile,blackbaloon,False,None,Being good pays back,"I am having a bad day, demons on my head are a...",https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-17 13:21:05
99913,MadeMeSmile,50EffingCabbages,False,None,My former mother in law just makes me happy now.,I married my first husband about thirty years ...,https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-18 01:56:15


In [15]:
# step6. final data quality check 

In [21]:
print(f'Author : {mademesmile_submissions.iloc[[6],:].author.tolist()[0]}')
print(f'Subreddit : {mademesmile_submissions.iloc[[6],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {mademesmile_submissions.iloc[[6],:].over_18.tolist()[0]}')
print(f'Posted : {mademesmile_submissions.iloc[[6],:].created_utc.tolist()[0]}')
print()
print(f'Title : {mademesmile_submissions.iloc[[6],:].title.tolist()[0]}')
print(f'Body : {mademesmile_submissions.iloc[[6],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {mademesmile_submissions.iloc[[7],:].author.tolist()[0]}')
print(f'Subreddit : {mademesmile_submissions.iloc[[7],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {mademesmile_submissions.iloc[[7],:].over_18.tolist()[0]}')
print(f'Posted : {mademesmile_submissions.iloc[[7],:].created_utc.tolist()[0]}')
print()
print(f'Title : {mademesmile_submissions.iloc[[7],:].title.tolist()[0]}')
print(f'Body : {mademesmile_submissions.iloc[[7],:].selftext.tolist()[0]}')


Author : InstructMeHow2Dougie
Subreddit : MadeMeSmile
Teenager(True/False) : False
Posted : 2013-02-03 07:21:19

Title : Teacher uses a shared interest in video games to connect with a troubled kid. (X-post from r/gaming)
Body : There was a kid in my class last semester, let's call him Matt. He has some troubles. He's an only child who lives out in the sticks, and from various comments he's made to me I gather that his family doesn't have much money. (The SPCA removed their horse because it was malnourished, for example.) He struggles in school and doesn't have many friends, if any. He's socially awkward, even compared to the average 14-year-old boy. He wears the same clothes everyday and they emit a distinctly foul odor. Halfway through last semester we were having a class talk about suicide and bullying since there had been a [relevant high-profile case](http://www.thestar.com/news/canada/2012/10/12/bc_victim_of_cyberbullying_commits_suicide.html) in the news. Matt broke down sobbing

In [22]:
# step7. save final pre-processed data into parquet
mademesmile_submissions.to_parquet('mademesmile_submissions_preprocessed.parquet', engine='pyarrow')
mademesmile_submissions_preprocessed = pd.read_parquet('mademesmile_submissions_preprocessed.parquet',engine='pyarrow')
mademesmile_submissions_preprocessed

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
34,MadeMeSmile,ProneToStanding,False,None,There's something about videos of dancing fool...,...so here's two examples!\n\nhttp://www.youtu...,http://www.reddit.com/r/MadeMeSmile/comments/y...,2012-08-10 18:57:24
48,MadeMeSmile,photo,False,None,Subreddit for 19 minutes!,This should catch on. 22 subscribers.,http://www.reddit.com/r/MadeMeSmile/comments/y...,2012-08-10 18:13:59
209,MadeMeSmile,gbaker386,False,None,"Gained faith in humanity, and it made my day",Sorry this is going to be a bit of a lengthy s...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-10-04 19:15:33
249,MadeMeSmile,[deleted],False,None,Helicopter rescues a RC plane from a tree,http://www.youtube.com/watch?v=ffEYqGGYXRk&amp...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-10-29 21:05:05
253,MadeMeSmile,K_Lobstah,False,None,"I normally hate cross-posting, but I thought y...",[Link to original post here](http://www.reddit...,http://www.reddit.com/r/MadeMeSmile/comments/1...,2012-11-02 19:44:20
...,...,...,...,...,...,...,...,...
99730,MadeMeSmile,jansbri,False,None,We Organized Carolers to Perform ‘All I Want F...,&amp;#x200B;\n\n[ ](https://reddit.com/link/eb...,https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-16 17:32:20
99814,MadeMeSmile,pokey_bum_wannk,False,None,"So, we have this Columbian fella working with us.",He doesn't speak much English but he's a real ...,https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-17 09:26:36
99831,MadeMeSmile,blackbaloon,False,None,Being good pays back,"I am having a bad day, demons on my head are a...",https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-17 13:21:05
99913,MadeMeSmile,50EffingCabbages,False,None,My former mother in law just makes me happy now.,I married my first husband about thirty years ...,https://www.reddit.com/r/MadeMeSmile/comments/...,2019-12-18 01:56:15
